In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [5]:
#ee.Authenticate()
#geemap.update_package()

ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

##Adding every plot coordinate
allplots_ga = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
allplots_va= pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Virginia/Coordinates and biomass/va_allpoints.csv')
allplots_va = allplots_va[allplots_va['liveMass'].notna()]
allplots_va = allplots_va[allplots_va['Latitude'].notna()]
allplots_va = allplots_va.drop(columns = ['deadMass', 'collectDate', 'totalMass', 'Ell', 'MSL', 'latitude', 'longitude'])
df_temp=allplots_va.copy()
df_temp = df_temp.groupby(['Year','locationID','Transect','Latitude','Longitude'],as_index = False).aggregate(
    {
        'liveMass':[np.mean, np.size]
    }
)
df_temp.columns = [
    'Year','locationID','Transect','Latitude','Longitude','liveMass','N'
]
allplots_va = df_temp
s_patens = pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Massachusetts/LTE-MP-LPP-biomass.csv')
s_alterniflora = pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Massachusetts/LTE-MP-LPA-biomass.csv')
s_patens['Latitude'] = 42.730953216553
s_patens['Longitude'] = -70.842918395996
s_alterniflora['Latitude'] = 42.731742858887
s_alterniflora['Longitude'] = -70.842468261719
s_patens.rename(columns={'LIVE biomass':'liveMass'}, inplace=True)
s_alterniflora.rename(columns={'MEAN BIOMASS':'liveMass'}, inplace=True)
df = pd.concat([s_patens, s_alterniflora])
df['Date'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])
dfx =  df.groupby(['SITE', 'Date'], as_index = False).aggregate(
    {
        'Longitude':[np.mean], 'Latitude':[np.mean], 'liveMass':[np.mean, np.size], 
        'MONTH':'first', 'YEAR':'first'
    }
) 
dfx.columns = [
    'Site','Date','Longitude','Latitude','liveMass', 'Sample size','Month', 'Year'
]

dfx.loc[dfx['Site'] == 'LTE-MP-LPP', 'Latitude'] = 42.730953216553
dfx.loc[dfx['Site'] == 'LTE-MP-LPP', 'Longitude'] = -70.842918395996
dfx.loc[dfx['Site'] == 'LTE-MP-LPA', 'Latitude'] = 42.731742858887
dfx.loc[dfx['Site'] == 'LTE-MP-LPA', 'Longitude'] = -70.842468261719

fc_ma = geemap.pandas_to_ee(dfx, latitude = 'Latitude', longitude = 'Longitude')
fc_va = geemap.pandas_to_ee(allplots_va, latitude = "Latitude", longitude = "Longitude")
fc_ga = geemap.csv_to_ee(allplots_ga, latitude = "Latitude", longitude = "Longitude")

Map.addLayer(fc_ma, {}, "MA")
Map.addLayer(fc_va, {}, "VA")
Map.addLayer(fc_ga, {}, "GA")

Map

Map(center=[31.539096, -81.422318], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [6]:
modis_ma = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(fc_ma) \
    .filter(ee.Filter.calendarRange(2012, 2012,'year'))

modis_ma = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(fc_va)
modis_ma = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(fc_ga)